In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from skl2onnx.helpers.onnx_helper import load_onnx_model
from stitchnet.stitchonnx.utils import Net
from glob import glob
from pathlib import Path

netsFiles = sorted(glob('_models/fragments/net*'))
nets = []
for i,netsFile in enumerate(netsFiles):
    fragmentFiles = sorted(glob(str(Path(netsFile)/'fragment*.onnx')))
    onnxFragments = []
    for fragmentFile in fragmentFiles:
        onnxFragment = load_onnx_model(fragmentFile)
        onnxFragments.append(onnxFragment)
    net1 = Net(onnxFragments, i)
    nets.append(net1)

/home/jupyter-steerapi/.conda/envs/stitchnet/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# import torch
# torch.ones(1,2).cuda()

In [4]:
from stitchnet.stitchonnx.utils import load_cats_and_dogs_dset,convert_imagenet_to_cat_dog_label
from stitchnet.stitchonnx.utils import accuracy_score_model,accuracy_score_net,load_dl
from stitchnet.stitchonnx.utils import generate_networks, ScoreMapper
from stitchnet.stitchonnx.report import Report
from stitchnet.stitchonnx.utils import evalulate_stitchnet

from tqdm import tqdm
import torch
import numpy as np
import os
from collections import defaultdict
import hashlib
import random
import time

random.seed(51)
np.random.seed(24)
torch.manual_seed(77)

K = 2
STITCH_BATCH_SIZE = 32 # todo study the effect
MAX_DEPTH = 16
THRESOULD = 0
TOTAL_THRESOULD = 0.5

RESULT_NAME = f"{int(time.time())}_result_BS_{STITCH_BATCH_SIZE}_MD_{MAX_DEPTH}_T_{THRESOULD}_TT_{TOTAL_THRESOULD}_K_{K}"

EVAL_BATCH_SIZE = 64

dataset_train = load_cats_and_dogs_dset("train")
dataset_val = load_cats_and_dogs_dset("test")

dl_score = load_dl(dataset_train, STITCH_BATCH_SIZE)
data_score,t = next(iter(dl_score))
data_score = data_score.numpy()

In [5]:
# range(1)
k = 0
if os.path.exists(f'./_results/{RESULT_NAME}.txt'):
    with open(f'./_results/{RESULT_NAME}.txt', 'r') as f:
        k = len(f.read().split('\n'))

In [6]:
import torch
x = torch.ones(1024, 120)
W = torch.ones(48, 120)
# (1024x120 and 48x120)
x.matmul(W.T).shape

torch.Size([1024, 48])

In [12]:
from tqdm import tqdm

scoreMapper = ScoreMapper(nets, data_score)
with Report(EVAL_BATCH_SIZE, f'./_results/{RESULT_NAME}.txt', 'a') as report:
    # for _ in tqdm(range(50)):
    generator = generate_networks(nets, scoreMapper, data_score, 
                          threshold=THRESOULD, totalThreshold=TOTAL_THRESOULD, 
                          maxDepth=MAX_DEPTH, sample=False, K=K)
    for i,(s,net) in enumerate(generator):
        try:
            netname = f"_results/{RESULT_NAME}/net{k:03}"
            report.evaluate(nets, net, netname, s, dataset_val)
            net.save(netname)
            k += 1
        except Exception as e:
            print('ERROR', e)
            pass

current depth: 1
potential next fragments: 2
potential next fragments after thresholding of 0: 2 ['1.0', '0.64']
totalscore 1.0


Traceback (most recent call last):
  File "/tmp/ipykernel_1716713/3951805165.py", line 9, in <cell line: 4>
    for i,(s,net) in enumerate(generator):
  File "/home/jupyter-steerapi/stitchnet/stitchnet/stitchonnx/utils.py", line 1086, in generate_networks
    for score, curr in recursive_stitching(start, scoreMapper, data, threshold, totalThreshold, 1.0, maxDepth=maxDepth, sample=sample, K=K):
  File "/home/jupyter-steerapi/stitchnet/stitchnet/stitchonnx/utils.py", line 1066, in recursive_stitching
    newcurr_fragment = stitch_fragments(curr, nextf, data)
  File "/home/jupyter-steerapi/stitchnet/stitchnet/stitchonnx/utils.py", line 708, in stitch_fragments
    nw = adjust_w(tX, tY, w)
  File "/home/jupyter-steerapi/stitchnet/stitchnet/stitchonnx/utils.py", line 627, in adjust_w
    return adjust_w_conv(tX, tY, w)
  File "/home/jupyter-steerapi/stitchnet/stitchnet/stitchonnx/utils.py", line 615, in adjust_w_conv
    A = train_w(acts1, acts2, Ainit)
  File "/home/jupyter-steerapi/stitch

In [8]:
# dataset_val

In [11]:
print('DONE')

DONE
